<a 
    target="_blank" 
    href="https://colab.research.google.com/github/davidgasquez/gitcoin-grants-data-portal/blob/main/notebooks/sandbox.ipynb"> 
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Following parquet files can be accessed from `ipfs`, see the [**homepage**](/) for instructions.

Raw files are direct result of API fetch, while processed files have been cleaned and normalized. 

Current convention for processed files is to store `address` in lowercase format. Columns are renamed to `snake_case`. 

#### File listing

In [1]:
from IPython.display import Markdown, display


def md(content):
    display(Markdown(content))

In [2]:
# | output: asis

import os

# Make sure to `make export-db`
EXPORT_PATH = os.path.join("..", "data/tables")
EXTENSIONS = ".parquet"

raw_files = []
processed_files = []

for file in os.listdir(EXPORT_PATH):
    if file.endswith(EXTENSIONS):
        if file.startswith("public_raw"):
            raw_files.append(file)
        else:
            processed_files.append(file)


def print_section(section_header: str, file_list):
    md(f"**{section_header}**   ")
    for file in file_list:
        md(f"[{file}](#{file})  ")


raw_files = sorted(raw_files)
processed_files = sorted(processed_files)

print_section("Processed Files:", processed_files)
print()
print_section("Raw Files:", raw_files)

**Processed Files:**   

[application_answers.parquet](#application_answers.parquet)  

[chain_metadata.parquet](#chain_metadata.parquet)  

[passport_scores.parquet](#passport_scores.parquet)  

[projects.parquet](#projects.parquet)  

...

[public_raw_projects.parquet](#public_raw_projects.parquet)  

[public_raw_round_applications.parquet](#public_raw_round_applications.parquet)  

[public_raw_round_contributors.parquet](#public_raw_round_contributors.parquet)  

[public_raw_round_votes.parquet](#public_raw_round_votes.parquet)  

[public_raw_rounds.parquet](#public_raw_rounds.parquet)  

For each of the files you can find sample rows, and schema below.

## Processed Files

In [3]:
# | output: asis

import pandas as pd
from pyarrow import parquet as pq


def print_file_sections(files, with_schema=True):
    for file_name in files:
        file_path = os.path.join(EXPORT_PATH, file_name)
        if not os.path.exists(file_path):
            print(f"File not found: {file_name}")
            continue

        md(f"## [{file_name}](#file-listing)")

        if with_schema:
            try:
                schema = pq.read_schema(file_path)
                schema_str = str(schema).replace("\n", "<br>")
                md(schema_str)

            except Exception as e:
                print("Could not read schema:", e)

        display(pd.read_parquet(file_path).head(4))


print_file_sections(processed_files, with_schema=True)

## [application_answers.parquet](#file-listing)

question_id: string<br>project_id: string<br>application_id: int64<br>chain_id: string<br>round_id: string<br>question: string<br>question_type: string<br>answer: string

,question_id,project_id,application_id,chain_id,round_id,question,question_type,answer
0,1,0x27ecde7fbbd667c76299df57b340b18b025ceb0d5dd0...,0,280,0xfff13d967d27b564838d413d98a53e0c013fa7d9,Funding Sources,short-answer,ok
1,2,0x27ecde7fbbd667c76299df57b340b18b025ceb0d5dd0...,0,280,0xfff13d967d27b564838d413d98a53e0c013fa7d9,Team Size,number,123
2,1,0xae65357bd3576496fffa2f659acdd5681115d7799408...,0,280,0x2f9e1523fe85a26602bf6518f6dfb70d44721dc1,Funding Sources,short-answer,grants
3,2,0xae65357bd3576496fffa2f659acdd5681115d7799408...,0,280,0x2f9e1523fe85a26602bf6518f6dfb70d44721dc1,Team Size,number,2


## [chain_metadata.parquet](#file-listing)

name: string<br>short_name: string<br>info_url: string<br>chain_id: string<br>network_id: int64<br>native_currency: struct<name: string, symbol: string, decimals: int32><br>  child 0, name: string<br>  child 1, symbol: string<br>  child 2, decimals: int32<br>ens: struct<registry: string><br>  child 0, registry: string<br>explorers: list<element: map<string, string ('element')>><br>  child 0, element: map<string, string ('element')><br>      child 0, element: struct<key: string not null, value: string> not null<br>          child 0, key: string not null<br>          child 1, value: string<br>rpc: list<element: string><br>  child 0, element: string<br>parent: struct<type: string, chain: string, bridges: list<element: struct<url: string>>><br>  child 0, type: string<br>  child 1, chain: string<br>  child 2, bridges: list<element: struct<url: string>><br>      child 0, element: struct<url: string><br>          child 0, url: string

...

chain_id: string<br>round_id: string<br>id: string<br>transaction: string<br>block_number: int64<br>project_id: string<br>application_id: int64<br>voter: string<br>grant_address: string<br>token: string<br>amount: double<br>amount_usd: double<br>amount_round_token: double

,chain_id,round_id,id,transaction,block_number,project_id,application_id,voter,grant_address,token,amount,amount_usd,amount_round_token
0,250,0x19d84682c0fd83df5920e29987221fa71f4ea75b,0x39e17a8935624409da1f8932570c7d693a6328a6a42a...,0x31f54e74e1cc53ccea26462810ed32e3701901c8f64f...,67413218,0xd2d10ca4b6b87fb6c0052ca3e0748236363d9e961d70...,2,0xabf28f8d9adfb2255f4a059e37d3bce9104969db,0x37e5e6eb36467f98392ebbb35f110fbb8238e6d3,0x83791638da5eb2faa432aff1c65fba47c5d29510,3.000000e+18,2.999475,1.469453e+19
1,250,0x19d84682c0fd83df5920e29987221fa71f4ea75b,0xddc53d22fbf190f569b0eef91196ef00e5bda0ae4a80...,0x31f54e74e1cc53ccea26462810ed32e3701901c8f64f...,67413218,0x24472c9a83946f8694084da07f9ae3087499b299e7a2...,1,0xabf28f8d9adfb2255f4a059e37d3bce9104969db,0x37e5e6eb36467f98392ebbb35f110fbb8238e6d3,0x83791638da5eb2faa432aff1c65fba47c5d29510,1.000000e+19,9.998252,4.898177e+19
2,250,0x19d84682c0fd83df5920e29987221fa71f4ea75b,0x94951ac2dcab4019adf38e55a77238784a1cf725b653...,0x31f54e74e1cc53ccea26462810ed32e3701901c8f64f...,67413218,0x4821b33f7e2079a82acdbcc452a13a8d59cc33b6981b...,0,0xabf28f8d9adfb2255f4a059e37d3bce9104969db,0x37e5e6eb36467f98392ebbb35f110fbb8238e6d3,0x83791638da5eb2faa432aff1c65fba47c5d29510,2.000000e+19,19.996503,9.796354e+19
3,250,0x19d84682c0fd83df5920e29987221fa71f4ea75b,0x98ab19ff5a369c4a3d2ef8ae9fc546179b9135ca7012...,0x1285af31632b737edc3a2681840f6a454080f37fe93f...,67413524,0x4821b33f7e2079a82acdbcc452a13a8d59cc33b6981b...,0,0x37e5e6eb36467f98392ebbb35f110fbb8238e6d3,0x37e5e6eb36467f98392ebbb35f110fbb8238e6d3,0x83791638da5eb2faa432aff1c65fba47c5d29510,5.000000e+18,4.999126,2.449089e+19


## [rounds.parquet](#file-listing)

id: string<br>amount_usd: double<br>votes: int64<br>token: string<br>match_amount: double<br>match_amount_usd: double<br>unique_contributors: int64<br>application_meta_ptr: string<br>application_metadata: string<br>meta_ptr: string<br>metadata: string<br>applications_start_time: int64<br>applications_end_time: double<br>round_start_time: int64<br>round_end_time: double<br>created_at_block: int64<br>updated_at_block: int64<br>chain_id: string<br>name: string<br>round_type: string<br>program_address: string<br>sybil_defense: bool

,id,amount_usd,votes,token,match_amount,match_amount_usd,unique_contributors,application_meta_ptr,application_metadata,meta_ptr,...,applications_end_time,round_start_time,round_end_time,created_at_block,updated_at_block,chain_id,name,round_type,program_address,sybil_defense
0,0xf16c7c3e7bb378c716f2b349a999d8ce56f3cfb5,0.010547,3,0x0000000000000000000000000000000000000000,1.000000e+18,0.525015,2,bafkreid4qdvpgogw7pec5m57uvdurkvdu5z7vmtuczuoa...,"{'lastUpdatedOn': 1695307508483, 'applicationS...",bafkreid3oppvwhxli3sqkf3br2b3k4rwa74uczwl4v2ii...,...,1.695341e+09,1695307560,1.695341e+09,47813168,47813168,137,POLYGON ROUND,public,0xf0da973d2610aee80265a614c0b5f43dedaf0e24,True
1,0xb410680b922e91f2ea4159331ffeaaadb217f246,0.000000,0,0x0000000000000000000000000000000000000000,1.000000e+18,0.526396,0,bafkreie2uqqanbeiuabkrzpghjqkjlgimjbm5w3t5yver...,"{'lastUpdatedOn': 1695318545342, 'applicationS...",bafkreifaysglowdjlhivjv5utus766tos4me3zroygbcy...,...,1.696032e+09,1695319200,1.696032e+09,47818260,47818260,137,Michelle's Test Round,private,0x2707d4017f72e2a4e688ed18b4a9b16fbb8adf8d,True
2,0x353d3485fb5b560bcb79780ced5ddd26331040a9,0.000000,0,0x0000000000000000000000000000000000000000,3.000000e+18,1.575025,0,bafkreifiyr7moy45suwlu7wvpa3ftdxhsgaxsbw2m3grx...,"{'lastUpdatedOn': 1695328536702, 'applicationS...",bafkreifbwivdn25q7tnkecu6wt6nzutndyb25w5zpbfre...,...,1.695686e+09,1695600000,1.695686e+09,47822940,47822940,137,July Testing Round,private,0x0b36268807abcdffe7333331990c90c271ef5754,True
3,0xbe245931c95d9b005ebf613090d932d10d3d5249,0.000000,0,0x0000000000000000000000000000000000000000,5.000000e+23,260098.220000,0,bafkreih37fpyqtpxrigid7ceobwrknd5wo3zfvwi4cqrs...,"{'lastUpdatedOn': 1695486571265, 'applicationS...",bafkreidetdecljkkzkpkwybvhjflne7g6eugotiix5cre...,...,1.698710e+09,1695859200,1.698710e+09,47895093,47895093,137,Join the African Revolution,public,0x28b4b1e3b3170f8a956f7202b5484db6c1492e7f,False


## Raw Files

In [4]:
# | echo: false
# | output: asis
print_file_sections(raw_files, with_schema=True)

## [public_raw_chain_metadata.parquet](#file-listing)

name: string<br>chain: string<br>icon: string<br>rpc: list<element: string><br>  child 0, element: string<br>features: list<element: struct<name: string>><br>  child 0, element: struct<name: string><br>      child 0, name: string<br>faucets: list<element: int32><br>  child 0, element: int32<br>nativeCurrency: struct<name: string, symbol: string, decimals: int32><br>  child 0, name: string<br>  child 1, symbol: string<br>  child 2, decimals: int32<br>infoURL: string<br>shortName: string<br>chainId: string<br>networkId: int64<br>slip44: int64<br>ens: struct<registry: string><br>  child 0, registry: string<br>explorers: list<element: map<string, string ('element')>><br>  child 0, element: map<string, string ('element')><br>      child 0, element: struct<key: string not null, value: string> not null<br>          child 0, key: string not null<br>          child 1, value: string<br>title: string<br>status: string<br>redFlags: int32<br>parent: struct<type: string, chain: string, bridges: list<element: struct<url: string>>><br>  child 0, type: string<br>  child 1, chain: string<br>  child 2, bridges: list<element: struct<url: string>><br>      child 0, element: struct<url: string><br>          child 0, url: string

,name,chain,icon,rpc,features,faucets,nativeCurrency,infoURL,shortName,chainId,networkId,slip44,ens,explorers,title,status,redFlags,parent
0,Ethereum Mainnet,ETH,ethereum,[https://mainnet.infura.io/v3/${INFURA_API_KEY...,"[{'name': 'EIP155'}, {'name': 'EIP1559'}]",[],"{'name': 'Ether', 'symbol': 'ETH', 'decimals':...",https://ethereum.org,eth,1,1,60.0,{'registry': '0x00000000000C2E074eC69A0dFb2997...,"[[(name, etherscan), (url, https://etherscan.i...",None,None,NaN,None
1,OP Mainnet,ETH,None,"[https://mainnet.optimism.io, https://optimism...",None,[],"{'name': 'Ether', 'symbol': 'ETH', 'decimals':...",https://optimism.io,oeth,10,10,NaN,None,"[[(name, etherscan), (url, https://optimistic....",None,None,NaN,None
2,Polygon Mainnet,Polygon,polygon,"[https://polygon-rpc.com/, https://rpc-mainnet...",None,[],"{'name': 'MATIC', 'symbol': 'MATIC', 'decimals...",https://polygon.technology/,matic,137,137,966.0,None,"[[(name, polygonscan), (url, https://polygonsc...",None,None,NaN,None
3,Fantom Opera,FTM,fantom,"[https://rpc.ftm.tools, https://fantom.publicn...",None,[],"{'name': 'Fantom', 'symbol': 'FTM', 'decimals'...",https://fantom.foundation,ftm,250,250,NaN,None,"[[(name, ftmscan), (url, https://ftmscan.com),...",None,None,NaN,None


## [public_raw_passport_scores.parquet](#file-listing)

address: string<br>score: int64<br>last_score_timestamp: timestamp[us, tz=UTC]<br>status: string<br>evidence: struct<type: string, success: bool, rawScore: string, threshold: string><br>  child 0, type: string<br>  child 1, success: bool<br>  child 2, rawScore: string<br>  child 3, threshold: string<br>id: int64

...

chainId: string<br>roundId: string<br>id: string<br>transaction: string<br>blockNumber: int64<br>projectId: string<br>applicationId: int64<br>voter: string<br>grantAddress: string<br>token: string<br>amount: double<br>amountUSD: double<br>amountRoundToken: double

,chainId,roundId,id,transaction,blockNumber,projectId,applicationId,voter,grantAddress,token,amount,amountUSD,amountRoundToken
0,250,0x19D84682c0FD83DF5920e29987221fA71f4EA75B,0x39e17a8935624409da1f8932570c7d693a6328a6a42a...,0x31f54e74e1cc53ccea26462810ed32e3701901c8f64f...,67413218,0xd2d10ca4b6b87fb6c0052ca3e0748236363d9e961d70...,2,0xABF28f8D9adFB2255F4a059e37d3BcE9104969dB,0x37e5e6eb36467F98392ebBb35F110FbB8238e6D3,0x83791638da5EB2fAa432aff1c65fbA47c5D29510,3.000000e+18,2.999475,1.469453e+19
1,250,0x19D84682c0FD83DF5920e29987221fA71f4EA75B,0xddc53d22fbf190f569b0eef91196ef00e5bda0ae4a80...,0x31f54e74e1cc53ccea26462810ed32e3701901c8f64f...,67413218,0x24472c9a83946f8694084da07f9ae3087499b299e7a2...,1,0xABF28f8D9adFB2255F4a059e37d3BcE9104969dB,0x37e5e6eb36467F98392ebBb35F110FbB8238e6D3,0x83791638da5EB2fAa432aff1c65fbA47c5D29510,1.000000e+19,9.998252,4.898177e+19
2,250,0x19D84682c0FD83DF5920e29987221fA71f4EA75B,0x94951ac2dcab4019adf38e55a77238784a1cf725b653...,0x31f54e74e1cc53ccea26462810ed32e3701901c8f64f...,67413218,0x4821b33f7e2079a82acdbcc452a13a8d59cc33b6981b...,0,0xABF28f8D9adFB2255F4a059e37d3BcE9104969dB,0x37e5e6eb36467F98392ebBb35F110FbB8238e6D3,0x83791638da5EB2fAa432aff1c65fbA47c5D29510,2.000000e+19,19.996503,9.796354e+19
3,250,0x19D84682c0FD83DF5920e29987221fA71f4EA75B,0x98ab19ff5a369c4a3d2ef8ae9fc546179b9135ca7012...,0x1285af31632b737edc3a2681840f6a454080f37fe93f...,67413524,0x4821b33f7e2079a82acdbcc452a13a8d59cc33b6981b...,0,0x37e5e6eb36467F98392ebBb35F110FbB8238e6D3,0x37e5e6eb36467F98392ebBb35F110FbB8238e6D3,0x83791638da5EB2fAa432aff1c65fbA47c5D29510,5.000000e+18,4.999126,2.449089e+19


## [public_raw_rounds.parquet](#file-listing)

id: string<br>amountUSD: double<br>votes: int64<br>token: string<br>matchAmount: double<br>matchAmountUSD: double<br>uniqueContributors: int64<br>applicationMetaPtr: string<br>applicationMetadata: string<br>metaPtr: string<br>metadata: string<br>applicationsStartTime: int64<br>applicationsEndTime: double<br>roundStartTime: int64<br>roundEndTime: double<br>createdAtBlock: int64<br>updatedAtBlock: int64<br>chainId: string

,id,amountUSD,votes,token,matchAmount,matchAmountUSD,uniqueContributors,applicationMetaPtr,applicationMetadata,metaPtr,metadata,applicationsStartTime,applicationsEndTime,roundStartTime,roundEndTime,createdAtBlock,updatedAtBlock,chainId
0,0xf16C7C3e7bb378c716F2b349a999D8ce56F3cFb5,0.010547,3,0x0000000000000000000000000000000000000000,1.000000e+18,0.525015,2,bafkreid4qdvpgogw7pec5m57uvdurkvdu5z7vmtuczuoa...,"{'lastUpdatedOn': 1695307508483, 'applicationS...",bafkreid3oppvwhxli3sqkf3br2b3k4rwa74uczwl4v2ii...,"{""feesPercentage"": 0, ""feesAddress"": """", ""name...",1695307560,1.695341e+09,1695307560,1.695341e+09,47813168,47813168,137
1,0xb410680b922e91f2eA4159331FFEAAAdb217F246,0.000000,0,0x0000000000000000000000000000000000000000,1.000000e+18,0.526396,0,bafkreie2uqqanbeiuabkrzpghjqkjlgimjbm5w3t5yver...,"{'lastUpdatedOn': 1695318545342, 'applicationS...",bafkreifaysglowdjlhivjv5utus766tos4me3zroygbcy...,"{""feesPercentage"": 0, ""feesAddress"": """", ""name...",1695319200,1.696032e+09,1695319200,1.696032e+09,47818260,47818260,137
2,0x353d3485Fb5b560BcB79780CEd5Ddd26331040a9,0.000000,0,0x0000000000000000000000000000000000000000,3.000000e+18,1.575025,0,bafkreifiyr7moy45suwlu7wvpa3ftdxhsgaxsbw2m3grx...,"{'lastUpdatedOn': 1695328536702, 'applicationS...",bafkreifbwivdn25q7tnkecu6wt6nzutndyb25w5zpbfre...,"{""feesPercentage"": 0, ""feesAddress"": """", ""name...",1695589200,1.695686e+09,1695600000,1.695686e+09,47822940,47822940,137
3,0xBE245931C95d9B005ebf613090D932D10d3d5249,0.000000,0,0x0000000000000000000000000000000000000000,5.000000e+23,260098.220000,0,bafkreih37fpyqtpxrigid7ceobwrknd5wo3zfvwi4cqrs...,"{'lastUpdatedOn': 1695486571265, 'applicationS...",bafkreidetdecljkkzkpkwybvhjflne7g6eugotiix5cre...,"{""feesPercentage"": 0, ""feesAddress"": """", ""name...",1695661200,1.698710e+09,1695859200,1.698710e+09,47895093,47895093,137
